In [152]:
import pandas as pd
import re as r
import json
import numpy as np

In [18]:
with open('datasource\wikipedia-movies.json','r') as rfile:
    file = json.load(rfile)

In [20]:
wiki_raw = pd.DataFrame(file)

In [349]:
#inspect dataframe
wiki_raw.dtypes

url                  object
year                float64
imdb_link            object
title                object
Directed by          object
                     ...   
Russian              object
Hebrew               object
Revenue              object
Operating income     object
Polish               object
Length: 193, dtype: object

In [355]:
#print out null value in each column
[[column,wiki_raw[column].isnull().sum()] for column in wiki_raw.columns]

[['url', 158],
 ['year', 158],
 ['imdb_link', 211],
 ['title', 177],
 ['Directed by', 208],
 ['Produced by', 571],
 ['Screenplay by', 4987],
 ['Story by', 6306],
 ['Based on', 5112],
 ['Starring', 391],
 ['Narrated by', 7026],
 ['Music by', 808],
 ['Cinematography', 911],
 ['Edited by', 913],
 ['Productioncompany ', 2771],
 ['Distributed by', 712],
 ['Release date', 397],
 ['Running time', 342],
 ['Country', 585],
 ['Language', 438],
 ['Budget', 2536],
 ['Box office', 1781],
 ['Written by', 2714],
 ['Genre', 7182],
 ['Theme music composer', 7241],
 ['Country of origin', 7153],
 ['Original language(s)', 7147],
 ['Producer(s)', 7150],
 ['Editor(s)', 7160],
 ['Production company(s)', 7175],
 ['Original network', 7180],
 ['Original release', 7146],
 ['Productioncompanies ', 6582],
 ['Executive producer(s)', 7210],
 ['Production location(s)', 7261],
 ['Distributor', 7171],
 ['Picture format', 7242],
 ['Audio format', 7247],
 ['Voices of', 7308],
 ['Followed by', 7297],
 ['Composer(s)', 7300

In [357]:
# get ride of columns that have too much null value is under 90% of total, there are 22 columns left
wiki_raw1 =wiki_raw[[column for column in wiki_raw.columns if wiki_raw[column].isnull().sum() < 0.9*len(wiki_raw)]]
wiki_raw1

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Edited by,Productioncompany,Distributed by,Release date,Running time,Country,Language,Budget,Box office,Written by
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,Michael Tronick,Silver Pictures,20th Century Fox,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,United States,English,$20 million,$21.4 million,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,Howard E. Smith,Avenue Pictures,Avenue Pictures,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,United States,English,$6 million,$2.7 million,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,"[John Bloom, Lois Freeman-Fox]","[Carolco Pictures, IndieProd Company]",TriStar Pictures,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,United States,"[English, Lao]",$35 million,"$57,718,089",NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,Susan E. Morse,NaN,Orion Pictures,"[December 25, 1990, (, 1990-12-25, )]",106 minutes,United States,English,$12 million,"$7,331,647",Woody Allen
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,David Stiven,NaN,Paramount Pictures,"December 19, 1990",95 minutes,US,English,$25 million,"$6,939,946 (USA)",Paul Hogan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7306,https://en.wikipedia.org/wiki/Holmes_%26_Watson,2018.0,https://www.imdb.com/title/tt1255919/,Holmes & Watson,Etan Cohen,"[Will Ferrell, Adam McKay, Jimmy Miller, Clayt...",Etan Cohen,NaN,"[Sherlock Holmes, and, Dr. Watson, by, Sir Art...","[Will Ferrell, John C. Reilly, Rebecca Hall, R...",...,Dean Zimmerman,NaN,Sony Pictures Releasing,"[December 25, 2018, (, 2018-12-25, ), (United ...",90 minutes,United States,English,$42 million,$41.9 million,NaN
7307,https://en.wikipedia.org/wiki/Vice_(2018_film),2018.0,https://www.imdb.com/title/tt6266538/,Vice,Adam McKay,"[Brad Pitt, Dede Gardner, Jeremy Kleiner, Kevi...",NaN,NaN,NaN,"[Christian Bale, Amy Adams, Steve Carell, Sam ...",...,Hank Corwin,"[Plan B Entertainment, Gary Sanchez Production...",Mirror Releasing,"[December 11, 2018, (, 2018-12-11, ), (, Samue...",132 minutes,United States,English,$60 million,$76.1 million,Adam McKay
7308,https://en.wikipedia.org/wiki/On_the_Basis_of_Sex,2018.0,https://www.imdb.com/title/tt4669788/,On the Basis of Sex,Mimi Leder,Robert W. Cort,NaN,NaN,NaN,"[Felicity Jones, Armie Hammer, Justin Theroux,...",...,Michelle Tesoro,NaN,Focus Features,"[November 8, 2018, (, 2018-11-08, ), (, AFI Fe...",120 minutes,United States,English,$20 million,$38.4 million,Daniel Stiepleman
7309,https://en.wikipedia.org/wiki/Destroyer_(2018_...,2018.0,https://www.imdb.com/title/tt7137380/,Destroyer,Karyn Kusama,"[Fred Berger, Phil Hay, Matt Manfredi]",NaN,NaN,NaN,"[Nicole Kidman, Sebastian Stan, Toby Kebbell, ...",...,Plummy Tucker,"[30West, Automatik Entertainment, Annapurna Pi...",Mirror Releasing,"[August 31, 2018, (, 2018-08-3

In [359]:
# data type inspection 
wiki_raw1.dtypes
# columns such as box office budget running time should be numbers

url                    object
year                  float64
imdb_link              object
title                  object
Directed by            object
Produced by            object
Screenplay by          object
Story by               object
Based on               object
Starring               object
Music by               object
Cinematography         object
Edited by              object
Productioncompany      object
Distributed by         object
Release date           object
Running time           object
Country                object
Language               object
Budget                 object
Box office             object
Written by             object
dtype: object

In [367]:
# for box office,1st get ride of null value
box_office = wiki_raw1['Box office']
box_office = box_office[box_office.notnull()]
Counter(box_office.map(type))

Counter({str: 5393, list: 137})

In [374]:
# there are list in this column, astype list
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
Counter(box_office.map(type))


Counter({str: 5530})

In [379]:
# inspect str data in box_office
box_office[:60]
# the first 100 have million, number format with thousand seperator, regular express to change the format

0                        $21.4 million
1                         $2.7 million
2                          $57,718,089
3                           $7,331,647
4                     $6,939,946 (USA)
9                             $855,810
10                      $195.3 million
11                       $53.2 million
12                       $15.7 million
13                       $52.1 million
14                      $244.5 million
15                         $12,626,043
17                         $19,740,070
18                        $138,697,012
20                          $1,292,323
21                        $8.2 million
22                       $15.6 million
24                          $2,070,871
25       $27.6 million (United States)
27                             $10,173
29                          $5 million
33                       $35.8 million
34                       US$ 4,212,828
37                        $2.4 million
39                            $947,306
40                       

In [387]:
# regular expression 1st iteration
rformat = r'\$\d+[,\.]\d+\s*million'
formatone = box_office.str.contains(rformat,flags=r.IGNORECASE)
box_office.str.contains(rformat,flags=r.IGNORECASE).sum()

# 3303 value catched

3303

In [396]:
# now for those number donesn't have end up with million or billion
rformat2 = r'\$\d+([,\.]\d{3})+(?![mb]llion)'
formattwo = box_office.str.contains(rformat2,flags = r.IGNORECASE)
box_office[formattwo]

2            $57,718,089
3             $7,331,647
4       $6,939,946 (USA)
9               $855,810
15           $12,626,043
              ...       
7262            $260,136
7273            $100,116
7278             $19,996
7292            $756,473
7301      $1.148 billion
Name: Box office, Length: 1610, dtype: object

In [397]:
box_office[~formatone  &~formattwo]

29         $5 million
34      US$ 4,212,828
60       $240 million
66        $86 million
68        $25 million
            ...      
7269     $174 million
7281      $76 million
7286     $323 million
7291      $43 million
7302     $468 million
Name: Box office, Length: 620, dtype: object

In [398]:
# update format
rformat = r'\$\d+([,\.]\d+)*\s*million'
rformat2 = r'\$\d+([,\.]\d{3})+(?![mb]llion)'
formatone = box_office.str.contains(rformat,flags=r.IGNORECASE)
box_office.str.contains(rformat,flags=r.IGNORECASE).sum()

formattwo = box_office.str.contains(rformat2,flags = r.IGNORECASE)
box_office[~formatone  &~formattwo]

34                         US$ 4,212,828
113                   $4.35-4.37 million
133                        US$ 4,803,039
618                           $5000 (US)
756                         $ 11,146,270
993                             $ 50,004
1113                          35,254,617
1192    $ 407,618 (U.S.) (sub-total) [1]
1506                        $ 11,829,959
1541                          £3 million
1952                        ¥1.1 billion
2132                                 N/A
2193                                $309
2234               US$ 171.8 million [9]
2365                   US$ 3,395,581 [1]
2371            $ 1,223,034 ( domestic )
2757            $ 104,883 (US sub-total)
2784         926,423 admissions (France)
2817      $ 1.7 million (US) (sub-total)
3222           $32 [2] –33.1 million [1]
3774                                 TBA
4006                  $38.9–40.3 million
4026            CN¥3.650 million (China)
4270                          £7,385,434
4426            

In [405]:
# 3th iteration
rformat = r'\$\s*\d+([,\.]\d+)*\s*million'
rformat2 = r'\$\s*\d+([,\.]\d{3})+(?![mb]llion)'
formatone = box_office.str.contains(rformat,flags=r.IGNORECASE)
box_office.str.contains(rformat,flags=r.IGNORECASE).sum()

formattwo = box_office.str.contains(rformat2,flags = r.IGNORECASE)
box_office[~formatone  &~formattwo]

113              $4.35-4.37 million
618                      $5000 (US)
1113                     35,254,617
1541                     £3 million
1952                   ¥1.1 billion
2132                            N/A
2193                           $309
2784    926,423 admissions (France)
3222      $32 [2] –33.1 million [1]
3774                            TBA
4006             $38.9–40.3 million
4026       CN¥3.650 million (China)
4270                     £7,385,434
4426             $20.7–23.9 million
4476                         $20-30
4671                   $47.7 millon
4746        $45.2k (only in Turkey)
5645                          £2.56
5983                       413 733$
6222                        Unknown
6359             $17.5–18.4 million
6449             $41.8–41.9 million
6587                          $111k
6588                           $588
6813                 less than $372
7070                        8 crore
7133                    $6.9 millon
Name: Box office, dtype: obj

In [406]:
# take care of ---
box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

In [408]:
# drop all the left
rformat = r'\$\s*\d+([,\.]\d+)*\s*milli?on'
rformat2 = r'\$\s*\d+([,\.]\d{3})+(?![mb]lli?on)'
formatone = box_office.str.contains(rformat,flags=r.IGNORECASE)
box_office.str.contains(rformat,flags=r.IGNORECASE).sum()

formattwo = box_office.str.contains(rformat2,flags = r.IGNORECASE)
box_office[~formatone  &~formattwo]


618                      $5000 (US)
1113                     35,254,617
1541                     £3 million
1952                   ¥1.1 billion
2132                            N/A
2193                           $309
2784    926,423 admissions (France)
3774                            TBA
4026       CN¥3.650 million (China)
4270                     £7,385,434
4476                            $30
4746        $45.2k (only in Turkey)
5645                          £2.56
5983                       413 733$
6222                        Unknown
6587                          $111k
6588                           $588
6813                 less than $372
7070                        8 crore
Name: Box office, dtype: object

In [446]:
 def parse_dollars(s):
    # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

    # if input is of the form $###.# million
        if r.match(r'\$\s*\d+\.?\d*\s*milli?on',s,flags = r.IGNORECASE):
            # remove dollar sign and " million"
              s = r.sub('\$|\s|[a-zA-Z]','',s)
            # convert to float and multiply by a million
              value = float(s) *10**6
            # return value
              return value
        # if input is of the form $###.# billion
        elif r.match(r'\$\s*\d+\.?\d*\s*billi?on',s,flags = r.IGNORECASE):

            # remove dollar sign and " billion"
              s = r.sub('\$|\s|[a-zA-Z]','',s)
            # convert to float and multiply by a billion
              value = float(s) *10**9
            # return value
              return value
        # if input is of the form $###,###,###
        elif r.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)',s,r.IGNORECASE):
            # remove dollar sign and commas
              s = r.sub('\$|\s|\.|,|[a-zA-Z]','',s)
            # convert to float
              value = float(s)
            # return value
              return value
        # otherwise, return NaN
        else:
            return np.nan
box_office.str.extract(f'({rformat}|{rformat2})',flags = r.IGNORECASE)[0].apply(parse_dollars)

0       21400000.0
1        2700000.0
2       57718089.0
3        7331647.0
4        6939946.0
           ...    
7305    19400000.0
7306    41900000.0
7307    76100000.0
7308    38400000.0
7309     5500000.0
Name: 0, Length: 5530, dtype: float64

In [447]:
box_office.str.extract(f'({rformat}|{rformat2})',flags = r.IGNORECASE)[0].apply(parse_dollars)[:50]

0      21400000.0
1       2700000.0
2      57718089.0
3       7331647.0
4       6939946.0
9        855810.0
10    195300000.0
11     53200000.0
12     15700000.0
13     52100000.0
14    244500000.0
15     12626043.0
17     19740070.0
18    138697012.0
20      1292323.0
21      8200000.0
22     15600000.0
24      2070871.0
25     27600000.0
27        10173.0
29      5000000.0
33     35800000.0
34      4212828.0
37      2400000.0
39       947306.0
40       715983.0
42     13236513.0
43      8300000.0
44      1373728.0
45    424200000.0
48     48800000.0
50    157900000.0
55      2200000.0
56      6698361.0
58      2700000.0
59    162700000.0
60    240000000.0
63      2346150.0
65     18100000.0
66     86000000.0
68     25000000.0
69      1372350.0
70     39000000.0
75     14760451.0
76     22424195.0
77      6488144.0
78     61300000.0
79      1823154.0
80      1305887.0
81       205068.0
Name: 0, dtype: float64

In [437]:
r.match(r'\$\s*\d+\.?\d*\s*milli?on','$21.4 million',r.IGNORECASE) == False

False

In [445]:
s = r.sub('\$|\s|\.|,|[a-zA-Z]','',box_office[2])
s

'57718089'